## 참고자료

1. https://colab.research.google.com/gist/SauravMaheshkar/168f0817f0cd29dd4048868fb0dd4401/lstms-in-pytorch.ipynb#scrollTo=n1XaWIg0w6XZ
2.
https://wikidocs.net/60691

# 라이브러리 import

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torchtext import data
from torch.autograd import Variable
from torchtext import data, datasets
from torchtext.vocab import Vectors, GloVe
import os

/home/dsail/hyolim/hyolim_py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
learning_rate = 2e-5
batch_size = 32
output_size = 2
hidden_size = 256
embedding_length = 300
epochs = 10

# 전처리

In [3]:
def load_dataset(test_sen=None):
    
    tokenize = lambda x: x.split()
    TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True, batch_first=True, fix_length=200)
    LABEL = data.LabelField()
    train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
    TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))
    LABEL.build_vocab(train_data)

    word_embeddings = TEXT.vocab.vectors

    train_data, valid_data = train_data.split()
    train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), 
                                                                   batch_size=32, 
                                                                   sort_key=lambda x: len(x.text), 
                                                                   repeat=False, shuffle=True)

    vocab_size = len(TEXT.vocab)

    return TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter

TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter = load_dataset()

* 필드 정의하기(torchtext.data)
    * 앞으로 어떤 전처리를 할 것인지를 정의
    * tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
    * batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값), 신경망에 입력되는 텐서의 첫번째 차원 값이 batch_size가 되게 함

- 'TEXT'의 경우, 데이터셋이 순차적인 데이터이므로 sequential=True설정
- 'LABEL'의 경우 단순한 클래스를 나타내는 숫자로 순차적 데이터가 아니므로 False로 명시

In [4]:
tokenize = lambda x: x.split() # 문장 토큰화 하기

TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, 
                  include_lengths=True, batch_first=True, fix_length=200)
LABEL = data.LabelField(sequential=False, batch_first=True)

# 데이터셋 로드 및 분할

In [5]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [6]:
print(vars(train_data[0]))

{'text': ['a', 'doctor', 'and', 'a', 'policeman', 'in', 'new', 'orleans', 'have', 'only', '48', 'hours', 'to', 'locate', 'a', 'killer', 'infected', 'with', 'pneumonic', 'plague.<br', '/><br', '/>an', 'effective,', 'and', 'classy,', 'little', 'thriller', 'directed', 'by', 'elia', 'kazan', 'that', 'blends', 'documentary', 'realism', 'with', 'a', 'race', 'against', 'time', 'pulpy', 'heartbeat.', 'set', 'and', 'filmed', 'in', 'and', 'around', 'new', 'orleans,', 'panic', 'in', 'the', 'streets', 'is', 'taken', 'from', 'the', 'story', 'quarantine,', 'some', 'like', "'em", 'cold', 'by', 'edna', 'and', 'edward', 'anhalt', 'who', 'won', 'an', 'oscar', 'for', 'original', 'story.', 'it', 'also', 'boasts', 'a', 'fine', 'ensemble', 'cast', 'that', 'deliver', 'top', 'rate', 'performances', 'for', 'their', 'director.', 'in', 'turn,', 'richard', 'widmark', '{bringing', 'the', 'method', 'a', 'year', 'before', 'marlon', 'did', 'for', 'kazan', 'in', 'a', 'streetcar', 'named', 'desire},', 'paul', 'douglas,

- 'text' : []에서 대괄호 안에 위치한 단어들이 첫번째 IMDB 리뷰
- 'label' : []에서 대괄호 안의 단어가 첫번째 IMDB 리뷰의 레이블. neg는 부정

# 단어집합 만들기
* 단어집합: 중복을 제거한 총 단어들의 집합

In [7]:
TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))
LABEL.build_vocab(train_data)

In [8]:
# 단어 집합의 크기: 중복을 제거한 총 단어의 개수
vocab_size = len(TEXT.vocab)
vocab_size

251639

In [9]:
word_embeddings = TEXT.vocab.vectors
print(word_embeddings.shape)
word_embeddings 

torch.Size([251639, 300])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

# 데이터 로더 만들기

In [10]:
train_data, valid_data = train_data.split()
train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), 
                                                               batch_size=32, 
                                                               sort_key=lambda x: len(x.text), 
                                                               repeat=False, shuffle=True)

In [11]:
print('훈련 데이터의 미니 배치의 개수 : {}'.format(len(train_iter)))
print('테스트 데이터의 미니 배치의 개수 : {}'.format(len(test_iter)))
print('검증 데이터의 미니 배치의 개수 : {}'.format(len(valid_iter)))

훈련 데이터의 미니 배치의 개수 : 547
테스트 데이터의 미니 배치의 개수 : 782
검증 데이터의 미니 배치의 개수 : 235


In [12]:
for idx, batch in enumerate(train_iter):
    text = batch.text[0]
    target = batch.label
    target = torch.autograd.Variable(target).long()

- 배치의 크기 32 * 200
- fix_length=200으로 설정했기에, 배치 간 샘플 길이가 200임

In [13]:
text.shape

torch.Size([28, 200])

In [14]:
print(target.shape)
target

torch.Size([28])


tensor([1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
        1, 1, 1, 1])

In [15]:
input_sentence=text

# 모델 구현하기

In [16]:
class LSTMClassifier(nn.Module):
	def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
		super(LSTMClassifier, self).__init__()
		self.batch_size = batch_size
		self.output_size = output_size
		self.hidden_size = hidden_size
		self.vocab_size = vocab_size
		self.embedding_length = embedding_length
		
		self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
		self.word_embeddings.weight = nn.Parameter(weights, requires_grad=False) 
		self.lstm = nn.LSTM(embedding_length, hidden_size) # Our main hero for this tutorial
		self.label = nn.Linear(hidden_size, output_size)
		
	def forward(self, input_sentence, batch_size=None):
		input = self.word_embeddings(input_sentence) 
		input = input.permute(1, 0, 2) 
		if batch_size is None:
			h_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda()) 
			c_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda()) 
		else:
			h_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
			c_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
		output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
		final_output = self.label(final_hidden_state[-1]) 
		
		return final_output

def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):

In [17]:
batch_size = batch_size
output_size = output_size
hidden_size = hidden_size
vocab_size = vocab_size
embedding_length = embedding_length
word_embeddings = nn.Embedding(vocab_size, embedding_length) # Embedding(251639, 300)

In [18]:
weights= TEXT.vocab.vectors
word_embeddings.weight = nn.Parameter(weights, requires_grad=False) 
print(word_embeddings.weight.shape)
word_embeddings.weight

torch.Size([251639, 300])


Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [19]:
lstm = nn.LSTM(embedding_length, hidden_size) # LSTM(300, 256)
label = nn.Linear(hidden_size, output_size) # Linear(in_features=256, out_features=2, bias=True)

def forward(self, input_sentence, batch_size=None):

In [20]:
input_sentence.shape

torch.Size([28, 200])

In [21]:
input = word_embeddings(input_sentence) # Embedding(251639, 300)
input.shape

torch.Size([28, 200, 300])

In [22]:
input = input.permute(1, 0, 2) 
input.shape

torch.Size([200, 28, 300])

Variable 클래스: 각 tensor의 값을 볼 수 있는 data, 미분을 보는 grad, backward를 통한 미분을 계산한 함수 정보인 grad_fn, 총 3개의 함수 사용 가능

In [23]:
h_0 = Variable(torch.zeros(1, batch_size, hidden_size))
print(h_0)
h_0.shape

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


torch.Size([1, 32, 256])

In [24]:
c_0 = Variable(torch.zeros(1, batch_size, hidden_size))
print(c_0)
c_0.shape

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


torch.Size([1, 32, 256])

In [25]:
output, (final_hidden_state, final_cell_state) = lstm(input, (h_0, c_0))
print(output.shape)
print(output)

RuntimeError: Expected hidden[0] size (1, 28, 256), got [1, 32, 256]

In [ ]:
(final_hidden_state, final_cell_state)
print(final_hidden_state.shape)
print(final_cell_state.shape)

In [ ]:
final_output = label(final_hidden_state[-1]) # label: Linear(in_features=256, out_features=2, bias=True)
final_output.shape